# Platform Support Agent

#### Install Dependencies

# State

In [1]:
from typing import TypedDict, List

class KnowledgeBase(TypedDict):
    knowledge: str
    source: str

class ServiceNowIncident(TypedDict):
    title: str
    description: str
    state: str
    id: str
    link: str

class InputState(TypedDict):
    problem: str

class ProcessingState(TypedDict):
    problem: str
    serviceNow: List[ServiceNowIncident]
    knowledgeBase: List[KnowledgeBase]
    solution: str
    search_result: List[str]
    errors: List[str]

class OutputState(TypedDict):
    solution: str
    source: List[str]


# Vector Store

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_nomic.embeddings import NomicEmbeddings


def query_vector_store(question):
    urls = [
        "https://lilianweng.github.io/posts/2023-06-23-agent/",
        "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
        "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
    ]

    # Load documents
    docs = [WebBaseLoader(url).load() for url in urls]
    docs_list = [item for sublist in docs for item in sublist]

    # Split documents
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=1000, chunk_overlap=200
    )
    doc_splits = text_splitter.split_documents(docs_list)

    # Add to vectorDB
    vectorstore = SKLearnVectorStore.from_documents(
        documents=doc_splits,
        embedding=NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local"),
    )
    # Create retriever
    retriever = vectorstore.as_retriever(k=3)
    docs = retriever.invoke(question)

    return docs




USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
from typing import List, Dict, Any
from langchain.agents import Tool
from langchain.schema import HumanMessage, AIMessage
from langchain_community.tools import DuckDuckGoSearchRun

from langchain_core.tools import tool

from langgraph.prebuilt import ToolNode

@tool
def get_knowledge(query: str) -> List[KnowledgeBase]:
    """
    Tool call to RAG service will return relevant information about the given keywords
    Args:
        query: query to get more information for 
    """
    try:
        return "Sample knowledge from knowledge server"
    except Exception as e:
        return [KnowledgeBase(knowledge=f"Error accessing knowledge base: {e}", source="")]


@tool
def get_knowledge_parallel(keywords: List[str]) -> List[KnowledgeBase]:
    """Tool call to RAG service will return relevant information about the given keywords"""
    # Implement parallel API calls here
    return [{"knowledge": "Sample knowledge from knowledge server", "source": "www.google.com"}]


@tool
def ddg_search(query: str) -> str:
    """
    duck_duck_go search tool call with a query about which you want to do web search
    Args:
        query: query to get more information for
    """
    try:
        search = DuckDuckGoSearchRun()
        response = search.run(query) # Use .run() instead of .invoke()
        return response
    except Exception as e:
        return f"Error performing DuckDuckGo search: {e}"


tools = [get_knowledge, get_knowledge_parallel]
tool_node = ToolNode(tools)

search_tools = [ddg_search]
search_tool_node = ToolNode(search_tools)


In [ ]:
"""Define a simple chatbot agent.

This agent returns a predefined response without using an actual LLM.
"""

from encodings import undefined
import json
from json import tool
from typing import Any, Dict, List, Literal, TypedDict

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig
from langchain_google_vertexai import ChatVertexAI
from langchain_core.output_parsers import JsonOutputParser
from langgraph.graph import StateGraph, END
from pydantic import conset
from langgraph.prebuilt import ToolNode
import os
from langchain_openai import AzureChatOpenAI, ChatOpenAI
from dotenv import load_dotenv
load_dotenv(override=True)
llm_azure = AzureChatOpenAI(
    azure_deployment="gpt-4o",
    api_version="2024-05-01-preview",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    verbose=True
    # organization="...",
    # model="gpt-35-turbo",
    # model_version="0125",
    # other params...
)
#  Initialize VertexAI model
llmPro = ChatVertexAI(
    model_name="gemini-2.0-pro-exp-02-05",  # You can change to another model as needed
    temperature=0,
    max_output_tokens=4096,
    project="sap-genai-playground-dev-mg",
    region="us-central1"
)

llmFlash = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

llm_with_tool = ChatVertexAI(
    model_name="gemini-1.5-flash",  # You can change to another model as needed
    temperature=0,
    max_output_tokens=4096,
    project="sap-genai-playground-dev-mg",
    region="us-central1"
).bind_tools(tools=tools)

llm_with_search = ChatVertexAI(
    model_name="gemini-1.5-flash",  # You can change to another model as needed
    temperature=0,
    max_output_tokens=4096,
    project="sap-genai-playground-dev-mg",
    region="us-central1"
).bind_tools(tools=search_tools)


def thinking(inputState: InputState) -> ProcessingState:
    """Thinking Node which will take user input and spend time reasoning it, analyzing requirements."""

    prompt = ChatPromptTemplate.from_messages([
        (
            "system",
            (
                "You are an analyst who has to identify the user's problem, after which you have to share the next steps "
                "that you need to take to fulfill the user's request. "
                "<example>"
                "I need to search the web to get information about <keyword>, I think the user is trying to achieve <this>"
                "</example>"
                "Please do not ask for additional information from the user. If there is any additional information required, "
                "then you have tool_calls which you can use, namely, knowledge_base and web_search. Mention you will have "
                "to use them if required in your thoughts."
            ),
        ),
        (
            "human",
            "Here is the problem: {prob}, return the output as JSONObject(problem: string)"
        ),
    ])
    chain =  prompt | llmFlash
    try:
        # Ensure the input matches the expected format
        print("Formatted Prompt: ", prompt.format(prob=inputState["problem"]))
        # Invoke the chain with the properly formatted input
        result = chain.invoke({"prob": inputState["problem"]})
        print("result of thinking: ", result)
        if not isinstance(result, AIMessage):
            raise ValueError("No message found in input")

        return {
            "search_result": [],
            "problem": result.content,
            "knowledgeBase": [],
            "serviceNow": [],
            "solution": "",
            "errors": []
        }
    except Exception as e:
        return {
            "search_result": [],
            "problem": inputState["problem"],
            "knowledgeBase": [],
            "serviceNow": [],
            "solution": "",
            "errors": [f"Error: {e}"]
        }

def should_get_knowledge(processingState: ProcessingState) -> str :
    if(processingState["knowledgeBase"] == [] or processingState["knowledgeBase"] == ""):
        return "tool_call"
    else:
        return "summarise"

def should_call_service_now(processingState: ProcessingState) -> str :
    if(processingState["knowledgeBase"] == []):
        return "summarise"
    else:
        return "service_now"


def knowledge_search(processingState: ProcessingState) -> ProcessingState:
    
    """Will search the Knowledge Base with a query about relevant keywords and get relevant information"""    

    prompt = ChatPromptTemplate.from_messages([
        ("system", """
         You are an analyst who have to identify user's problem and return answer as JSONObject(data: string, source: string)[]
         """),
        ("human", "Here are your thoughts on what you want to do: {input_source}\n Here is the context: {knowledge_base}")
    ])

    chain = prompt | llmFlash 
    
    try:
        result = chain.invoke({"input_source" : processingState["problem"], "knowledge_base": processingState["knowledgeBase"]})
        print("result of input_source: " , result)
        return {
            **processingState,
            "knowledgeBase": result.content,
            "errors": [],
        }
    except Exception as e:
        return {
            **processingState,
            "errors": [f"Error: {e}"]
        }
    

"""Will search the ServiceNOW with a query about relevant keywords and get relevant information"""
def service_now_search(processingState: ProcessingState) -> ProcessingState:
    
    """Will search the ServiceNOW with a query about relevant keywords and get relevant information"""    

    prompt = ChatPromptTemplate.from_messages([
        ("system", """
        Your job is to return the ServiceNOW Incidents as JSONObject(title: str, description: str, state: str, id: str, link: str)[]
        """),
        ("human", "Here are your thoughts on what you wanted to do: {input_source}\n Here are the incidents on ServiceNOW: {}")
    ])

    chain = prompt | llmFlash | JsonOutputParser()
    
    try:
        result = chain.invoke({"input_source" : processingState["problem"], "incidents": processingState["serviceNow"]})
        print("result of input_source: " , result)
        return {
            **processingState,
            "knowledgeBase": result,
            "serviceNow": [],
            "solution": "",
            "errors": [],
        }
    except Exception as e:
        return {
            **processingState,
            "knowledgeBase": [],
            "serviceNow": [],
            "solution": "",
            "errors": [f"Error: {e}"]
        }
    

def web_search(processingState: ProcessingState) -> ProcessingState:
    """
    Performs a web search using the duck_duck_go tool.
    """
    try:
        # Invoke the llm_with_search chain with the message
        result = llm_with_search.invoke(f"Gather information that you require based on: {processingState.problem}, based on this return a single line of search query") 

        print("result of web_search: ", result)  #Check the result

        search_result = ddg_search(result.content)

        return {
            **processingState,
            "search_result": [search_result]
        }
    except Exception as e:
        return {
            **processingState,
            "errors": [f"Error in web_search: {e}"],
        }

def should_search_web(processingState: ProcessingState) -> str :
    try:
        if(processingState["search_result"] == "" or len(processingState["search_result"]) == 0):
            return "duck_duck_go"
        else:
            return "knowledge_base"
    except:
        return "duck_duck_go"
        
def summarise(processingState: ProcessingState) -> OutputState:

    """Will search the Knowledge Base with a query about relevant keywords and get relevant information"""    

    prompt = ChatPromptTemplate.from_messages([
        ("system", """
        Give all the knowledge_base and search_results from suggest a solution to the user's problem, address the user directly and politely.
         """),
        ("human", "Here was the user's problem for which you thougt to do: {input_source}\n Here is the knowledge_base: {knowledge_base}\n give a solution with what steps the user can take in plain and understandable english")
    ])

    chain = prompt | llmPro
    
    try:
        result = chain.invoke({"input_source" : processingState["problem"], "knowledge_base": processingState["knowledgeBase"]})
        print("result of summarise: " , result.content)
        return {
            "source" : [],
            "solution" : result.content
        }
    except Exception as e:
        return {
            "source" : [],
            "solution" : f"Error occurred: {e}"
        }

tool_node = ToolNode(tools)

# Define a new graph
workflow = StateGraph(input=InputState, output=OutputState)

# Add the node to the graph
workflow.add_node(node="thinking",action=thinking)
workflow.add_node(node="knowledge_base", action=knowledge_search)
workflow.add_node(node="tool_call",action=tool_node)
workflow.add_node(node="web_search", action=web_search)
workflow.add_node(node="duck_duck_go", action=search_tool_node)
workflow.add_node(node="summarise", action=summarise)

# Set the entrypoint as `call_model`
workflow.set_entry_point("thinking")

# Define edges
workflow.add_edge("__start__", "thinking")
workflow.add_edge("thinking", "web_search")
workflow.add_edge(
    "web_search","knowledge_base"
)
workflow.add_conditional_edges(
    "knowledge_base"
    ,should_get_knowledge
    ,["tool_call","summarise"]
)
workflow.add_edge("tool_call", "knowledge_base")
workflow.add_edge("summarise", "__end__")


graph = workflow.compile()
graph.name = "Platform Support Agent"  # This defines the custom name in LangSmith


C:\Users\noepolak1\AppData\Local\Temp\ipykernel_10320\3746813217.py:248: LangGraphDeprecationWarning: Initializing StateGraph without state_schema is deprecated. Please pass in an explicit state_schema instead of just an input and output schema.
  workflow = StateGraph(input=InputState, output=OutputState)


In [5]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [6]:
input = {

    "problem": "I am getting an error while making network calls with broken pipe error"
}
result = graph.invoke(input)

Formatted Prompt:  System: You are an analyst who has to identify the user's problem, after which you have to share the next steps that you need to take to fulfill the user's request. <example>I need to search the web to get information about <keyword>, I think the user is trying to achieve <this></example>Please do not ask for additional information from the user. If there is any additional information required, then you have tool_calls which you can use, namely, knowledge_base and web_search. Mention you will have to use them if required in your thoughts.
Human: Here is the problem: I am getting an error while making network calls with broken pipe error, return the output as JSONObject(problem: string)
result of thinking:  content='I think the user is trying to address an issue related to network calls, specifically encountering a "broken pipe" error. The output they want is a JSON object that encapsulates the problem.\n\nNext, I will create a JSON object that includes the specified 

In [7]:
print(result["solution"])

Error occurred: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.


# Solution

### Solution Node

In [8]:
from IPython.display import display_markdown

display_markdown(result["solution"], raw=True)

Error occurred: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.